In [0]:
"""Notebook MVP 03 - Gerar tabela Silver (ciclos_referencia) para o racional.

Este notebook:
- Le o arquivo tratado `silver_ciclos_base.parquet` (Notebook 01).
- Aplica filtros previos (rodovia/sentido) para reduzir volume de dados.
- Aplica a logica do racional para selecionar 1 registro por:
    Data x Ciclo x KM x Rodovia x Sentido.
  (registro mais proximo do fim do ciclo, preferindo antes do limite).
- Calcula tambem a coluna `Minuto_no_Ciclo` (posicao dentro dos 180 min).
- Salva em `mvp/silver/silver_ciclos_referencia.parquet`.

Esta tabela sera a base para as agregacoes de conformidade na camada Gold.
"""

import os
from pathlib import Path

import numpy as np
import pandas as pd

# Usar caminhos do workspace diretamente
WORKSPACE_BASE = "/Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp"
SILVER_DIR = f"{WORKSPACE_BASE}/silver"

ARQUIVO_SILVER_BASE = f"{SILVER_DIR}/silver_ciclos_base.parquet"
ARQUIVO_SILVER_REF = f"{SILVER_DIR}/silver_ciclos_referencia.parquet"

# Filtros previos 
RODOVIA_TARGET = "BR-153/GO"  # ou None para processar todas
SENTIDO_TARGET = "S"          # ou None para processar ambos

# Tolerancias de tempo (aplicadas antes da selecao de referencia)
TOLERANCIA_TEMPO_ANTES = 15   # minutos antes do inicio do ciclo
TOLERANCIA_TEMPO_DEPOIS = 15  # minutos depois do fim do ciclo

print("=== CONFIGURACAO DE CAMINHOS ===")
print(f"Workspace base: {WORKSPACE_BASE}")
print(f"Diretorio Silver: {SILVER_DIR}")
print(f"Arquivo silver base: {ARQUIVO_SILVER_BASE}")
print(f"Arquivo silver referencia (saida): {ARQUIVO_SILVER_REF}")

print("\n=== CONFIGURACAO DE FILTROS ===")
print(f"Rodovia: {RODOVIA_TARGET if RODOVIA_TARGET else 'Todas'}")
print(f"Sentido: {SENTIDO_TARGET if SENTIDO_TARGET else 'Ambos'}")
print(f"Tolerancia tempo: -{TOLERANCIA_TEMPO_ANTES}/+{TOLERANCIA_TEMPO_DEPOIS} min")

# Verificar se arquivo existe
if not os.path.exists(ARQUIVO_SILVER_BASE):
    raise FileNotFoundError(
        f"silver_ciclos_base.parquet nao encontrado em {ARQUIVO_SILVER_BASE}\n"
        "Execute primeiro o notebook 01_gerar_silver_ciclos_base"
    )

print("\n=== CARREGANDO DADOS SILVER BASE ===")
print(f"Lendo silver_ciclos_base de: {ARQUIVO_SILVER_BASE}")
df = pd.read_parquet(ARQUIVO_SILVER_BASE)
print(f"[OK] Linhas carregadas: {len(df):,}")
print(f"Colunas: {list(df.columns)}")

# Aplicar filtros previos 
print("\n=== APLICANDO FILTROS PREVIOS ===")
if RODOVIA_TARGET:
    antes = len(df)
    df = df[df["Rodovia"] == RODOVIA_TARGET].copy()
    print(f"[OK] Após filtro rodovia ({RODOVIA_TARGET}): {len(df):,} linhas (removidas {antes - len(df):,})")

if SENTIDO_TARGET:
    antes = len(df)
    df = df[df["Sentido"] == SENTIDO_TARGET].copy()
    print(f"[OK] Após filtro sentido ({SENTIDO_TARGET}): {len(df):,} linhas (removidas {antes - len(df):,})")

# Garantir tipos corretos
if not np.issubdtype(df["Marcacao"].dtype, np.datetime64):
    df["Marcacao"] = pd.to_datetime(df["Marcacao"], errors="coerce")

# Recalcular Minutos e limites, se necessario
print("\n=== RECALCULANDO COLUNAS DERIVADAS ===")
df["Minutos"] = (
    df["Marcacao"].dt.hour * 60
    + df["Marcacao"].dt.minute
    + df["Marcacao"].dt.second / 60.0
)

df["Ciclo"] = df["Marcacao"].dt.hour // 3 + 1

df["Inicio_Ideal"] = (df["Ciclo"] - 1) * 180
df["Fim_Ideal"] = df["Ciclo"] * 180

# Aplicar filtro de tolerancia de tempo 
# Isso reduz significativamente o volume antes da selecao de referencia
print("\n=== APLICANDO FILTRO DE TOLERANCIA DE TEMPO ===")
df["Ini_OK"] = df["Inicio_Ideal"] - TOLERANCIA_TEMPO_ANTES
df["Fim_OK"] = df["Fim_Ideal"] + TOLERANCIA_TEMPO_DEPOIS

antes = len(df)
df = df[(df["Minutos"] >= df["Ini_OK"]) & (df["Minutos"] < df["Fim_OK"])].copy()
df = df.drop(columns=["Ini_OK", "Fim_OK"])

print(f"[OK] Após filtro de tolerancia de tempo: {len(df):,} linhas (removidas {antes - len(df):,})")

# Preferencia: 0 se passou antes (ou exatamente) do Fim_Ideal, 1 se apos
print("\n=== APLICANDO LOGICA DE SELECAO POR CICLO ===")
print("Preferencia: registros antes do fim do ciclo (preferencia=0)")

df["Preferencia"] = np.where(df["Minutos"] <= df["Fim_Ideal"], 0, 1)

df["Delta_limite"] = np.where(
    df["Preferencia"] == 0,
    df["Fim_Ideal"] - df["Minutos"],
    df["Minutos"] - df["Fim_Ideal"],
)

# Ordenar e selecionar 1 registro por Data x Ciclo x KM x Rodovia x Sentido
chaves = ["Data", "Ciclo", "Rodovia", "Sentido", "KM"]

colunas_ordenacao = chaves + ["Preferencia", "Delta_limite", "Marcacao"]

df_sorted = df.sort_values(colunas_ordenacao).copy()

# groupby().first() preserva o primeiro da ordenacao
print("Selecionando 1 registro por combinacao de chaves...")
print(f"Chaves de agrupamento: {chaves}")

df_ref = (
    df_sorted
    .groupby(chaves, as_index=False)
    .first()
)

print(f"[OK] Linhas na tabela de referencia: {len(df_ref):,}")

# Calcular posicao no ciclo (0..180) para uso na conformidade
print("\n=== CALCULANDO MINUTO_NO_CICLO ===")
df_ref["Minuto_no_Ciclo"] = df_ref["Minutos"] - df_ref["Inicio_Ideal"]

print("Resumo da posicao no ciclo (minutos):")
print(df_ref["Minuto_no_Ciclo"].describe())

print("\n=== SALVANDO DADOS SILVER REFERENCIA ===")
print(f"Salvando silver_ciclos_referencia.parquet em: {ARQUIVO_SILVER_REF}")

# Garantir que diretorio existe
os.makedirs(SILVER_DIR, exist_ok=True)

df_ref.to_parquet(ARQUIVO_SILVER_REF, index=False)

# Verificar se arquivo foi criado
if os.path.exists(ARQUIVO_SILVER_REF):
    tamanho_mb = os.path.getsize(ARQUIVO_SILVER_REF) / (1024 * 1024)
    print(f"[OK] Arquivo salvo com sucesso!")
    print(f"  Linhas na tabela Silver Referencia: {len(df_ref):,}")
    print(f"  Tamanho do arquivo: {tamanho_mb:.2f} MB")
    print(f"  Colunas: {len(df_ref.columns)}")
else:
    print("[ERRO] Arquivo nao foi criado!")

print("\n=== CONCLUSAO ===")
print("Camada Silver Referencia criada com sucesso!")
print(f"Arquivo: {ARQUIVO_SILVER_REF}")
print(f"\nResumo:")
print(f"  - Registros de entrada (apos filtros): {len(df):,}")
print(f"  - Registros de referencia (1 por Data/Ciclo/KM/Rodovia/Sentido): {len(df_ref):,}")
print(f"  - Reducao: {len(df) - len(df_ref):,} registros removidos")
print(f"\nProximo passo: Execute 03_gerar_gold_conformidade_racional")

=== CONFIGURACAO DE CAMINHOS ===
Workspace base: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp
Diretorio Silver: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/silver
Arquivo silver base: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/silver/silver_ciclos_base.parquet
Arquivo silver referencia (saida): /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/silver/silver_ciclos_referencia.parquet

=== CONFIGURACAO DE FILTROS ===
Rodovia: BR-153/GO
Sentido: S
Tolerancia tempo: -15/+15 min

=== CARREGANDO DADOS SILVER BASE ===
Lendo silver_ciclos_base de: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/silver/silver_ciclos_base.parquet
[OK] Linhas carregadas: 1,548,566
Colunas: ['Marcacao', 'Data', 'Rodovia', 'Sentido', 'KM', 'KM_original', 'Recurso', 'CodigoRecurso', 'Ciclo', 'Minutos', 'Inicio_Ideal', 'Fim_Ideal']

=== APLICANDO FILTROS PREVIOS ===
[OK] Após filtro rodovia (BR-153/GO): 872,597 linhas (removidas 675,969)
[OK] Após filtro sentido (S): 430,703 linhas 